# Training and Evaluation of GNNs and LLMs
In this notebook, we train the models on the [MovieLens Dataset](https://movielens.org/) after the Pytorch Geometrics Tutorial on [Link Prediction](https://colab.research.google.com/drive/1xpzn1Nvai1ygd_P5Yambc_oe4VBPK_ZT?usp=sharing#scrollTo=vit8xKCiXAue).

First we import all of our dependencies.

The **GraphRepresentationGenerator** manages and trains a GNN model. Its most important interfaces include
**the constructor**, which defines the GNN architecture and loads the pre-trained GNN model if it is already on the hard disk,
**the training method**, which initializes the training on the GNN model and
**the get_embedding methods**, which represent the inference interface to the GNN model and return the corresponding embeddings in the dimension defined in the constructor for given user movie node pairs.

**The MovieLensLoader** loads and manages the data sets. The most important tasks include **saving and (re)loading and transforming** the data sets.

**PromptEncoderOnlyClassifier** and **VanillaEncoderOnlyClassifier** each manage a **prompt (model) LLM** and a **vanilla (model) LLM**. An EncoderOnlyClassifier (ClassifierBase) provides interfaces for training and testing an LLM model.
PromptEncoder and VanillaEncoder differ from their DataCollectors. DataCollectors change the behavior of the models during training and testing and allow data points to be created at runtime. With the help of these collators, we **create non-existent edges on the fly**.

In [1]:
from graph_representation_generator import GraphRepresentationGenerator
from dataset_manager import (
    MovieLensManager,
    PROMPT_KGE_DIMENSION,
    INPUT_EMBEDS_REPLACE_KGE_DIMENSION,
    ROOT,
)
from llm_manager import (
    PromptBertClassifier,
    VanillaBertClassifier,
    InputEmbedsReplaceClassifier,
)

In [2]:
EPOCHS = 20
BATCH_SIZE = 256

We define in advance which **Knowledge Graph Embedding Dimension (KGE_DIMENSION)** the GNN encoder has. We want to determine from which output dimension the GNN encoder can produce embeddings that lead to a significant increase in performance *without exceeding the context length of the LLMs*. In the original tutorial, the KGE_DIMENSION was $64$.

In [3]:
kg_manager = MovieLensManager()

First we load the MovieLensLoader, which downloads the Movie Lens dataset (https://files.grouplens.org/datasets/movielens/ml-latest-small.zip) and prepares it to be used on GNN and LLM. We also pass the embedding dimensions that we will assume we are training with. First time takes approx. 30 sec.

In [4]:
kg_manager.data

HeteroData(
  source={ node_id=[610] },
  target={
    node_id=[9742],
    x=[9742, 20],
  },
  (source, edge, target)={ edge_index=[2, 100836] },
  (target, rev_edge, source)={ edge_index=[2, 100836] }
)

Next, we initialize the GNN trainers (possible on Cuda), one for each KGE_DIMENSION.
A GNN trainer manages a model and each model consists of an **encoder and classifier** part.

**The encoder** is a parameterized *Grap Convolutional Network (GCN)* with a *2-layer GNN computation graph* and a single *ReLU* activation function in between.

**The classifier** applies the dot-product between source and destination kges to derive edge-level predictions.

In [5]:
graph_representation_generator_prompt = GraphRepresentationGenerator(
    kg_manager.data,
    kg_manager.gnn_train_data,
    kg_manager.gnn_val_data,
    kg_manager.gnn_test_data,
    kge_dimension=PROMPT_KGE_DIMENSION,
)
graph_representation_generator_input_embeds_replace = GraphRepresentationGenerator(
    kg_manager.data,
    kg_manager.gnn_train_data,
    kg_manager.gnn_val_data,
    kg_manager.gnn_test_data,
    hidden_channels=INPUT_EMBEDS_REPLACE_KGE_DIMENSION,
    kge_dimension=INPUT_EMBEDS_REPLACE_KGE_DIMENSION,
)

loading pretrained model
Device: 'cuda'
loading pretrained model
Device: 'cuda'


We then train and validate the model on the link prediction task.

If the model is already trained, we can skip this part.
Training the models can take up to 5 minutes.

In [ ]:
print("Prompt Training")
graph_representation_generator_prompt.train_model(
    kg_manager.gnn_train_data, EPOCHS, BATCH_SIZE
)
graph_representation_generator_prompt.validate_model(kg_manager.gnn_test_data)
print("Attention Training")
graph_representation_generator_input_embeds_replace.train_model(
    kg_manager.gnn_train_data, EPOCHS, BATCH_SIZE
)
graph_representation_generator_input_embeds_replace.validate_model(
    kg_manager.gnn_test_data
)


Next we produce the KGEs for every edge in the dataset. These embeddings can then be used for the LLM on the link-prediction task.

In [ ]:
prompt_embeddings = graph_representation_generator_prompt.get_saved_embeddings("prompt")
input_embeds_replace_embeddings = (
    graph_representation_generator_input_embeds_replace.get_saved_embeddings(
        "input_embeds_replace"
    )
)
save = False
if prompt_embeddings is None or input_embeds_replace_embeddings is None:
    prompt_embeddings = graph_representation_generator_prompt.generate_embeddings(
        kg_manager.llm_df
    )
    input_embeds_replace_embeddings = (
        graph_representation_generator_input_embeds_replace.generate_embeddings(
            kg_manager.llm_df
        )
    )
    save = True

kg_manager.append_prompt_graph_embeddings(prompt_embeddings, save=save)
kg_manager.append_input_embeds_replace_graph_embeddings(
    input_embeds_replace_embeddings, save=save
)


In [ ]:
df = kg_manager.add_false_edges(
    1.0,
    graph_representation_generator_prompt.get_embedding,
    graph_representation_generator_input_embeds_replace.get_embedding,
    splits=["val", "test"],
)
df[df["split"] == "val"]

Next we initialize the vanilla encoder only classifier. This classifier does only use the NLP part of the prompt (no KGE) for predicting if the given link exists.

In [ ]:
VANILLA_ROOT = f"{ROOT}/llm/vanilla"

In [ ]:
vanilla_bert_classifier = VanillaBertClassifier(
    kg_manager.llm_df,
    kg_manager.source_df,
    kg_manager.target_df,
    root_path=VANILLA_ROOT,
)

Next we generate a vanilla llm dataset and tokenize it for training.

In [ ]:
dataset_vanilla = kg_manager.generate_vanilla_dataset(
    vanilla_bert_classifier.tokenize_function
)

Next we train the model on the produced dataset. This can be skipped, if already trained ones.

In [ ]:
vanilla_bert_classifier.train_model_on_data(
    dataset_vanilla, epochs=EPOCHS, batch_size=BATCH_SIZE
)

Next we initialize the prompt encoder only classifier. This classifier uses the vanilla prompt and the KGEs for its link prediction.

In [ ]:
PROMPT_ROOT = f"{ROOT}/llm/prompt"

In [ ]:
prompt_bert_classifier = PromptBertClassifier(
    kg_manager,
    graph_representation_generator_prompt.get_embedding,
    model_max_length=512,
    root_path=PROMPT_ROOT,
)

We also generate a prompt dataset, this time the prompts also include the KGEs.

In [ ]:
dataset_prompt = kg_manager.generate_prompt_embedding_dataset(
    prompt_bert_classifier.tokenize_function,
)

We also train the model. This can be skipped if already done ones.

In [ ]:
prompt_bert_classifier.train_model_on_data(
    dataset_prompt, epochs=EPOCHS, batch_size=BATCH_SIZE
)

In [ ]:
INPUT_EMBEDS_REPLACE_ROOT = f"{ROOT}/llm/input_embeds_replace"

In [ ]:
input_embeds_replace_bert_classifier = InputEmbedsReplaceClassifier(
    kg_manager,
    graph_representation_generator_input_embeds_replace.get_embedding,
    root_path=INPUT_EMBEDS_REPLACE_ROOT,
)

In [ ]:
dataset_embedding = kg_manager.generate_input_embeds_replace_embedding_dataset(
    input_embeds_replace_bert_classifier.tokenizer.sep_token,
    input_embeds_replace_bert_classifier.tokenizer.pad_token,
    input_embeds_replace_bert_classifier.tokenize_function,
)

In [ ]:
input_embeds_replace_bert_classifier.train_model_on_data(
    dataset_embedding, epochs=EPOCHS, batch_size=BATCH_SIZE
)

In [ ]:
kg_manager.add_false_edges(
    1.0,
    graph_representation_generator_prompt.get_embedding,
    graph_representation_generator_input_embeds_replace.get_embedding,
    splits=["train"],
)

In [ ]:
dataset_embedding

In [6]:
vanilla_bert_classifier = VanillaBertClassifier(
    kg_manager.llm_df,
    kg_manager.source_df,
    kg_manager.target_df,
    root_path=VANILLA_ROOT,
    false_ratio=-1,
)
prompt_bert_classifier = PromptBertClassifier(
    kg_manager,
    graph_representation_generator_prompt.get_embedding,
    root_path=PROMPT_ROOT,
    model_max_length=512,
    false_ratio=-1,
)
input_embeds_replace_bert_classifier = InputEmbedsReplaceClassifier(
    kg_manager,
    graph_representation_generator_input_embeds_replace.get_embedding,
    root_path=INPUT_EMBEDS_REPLACE_ROOT,
    false_ratio=-1,
)


In [7]:
dataset_vanilla = kg_manager.generate_vanilla_dataset(
    vanilla_bert_classifier.tokenize_function
)
dataset_prompt = kg_manager.generate_prompt_embedding_dataset(
    prompt_bert_classifier.tokenize_function,
)
dataset_input_embeds_replace = (
    kg_manager.generate_input_embeds_replace_embedding_dataset(
        input_embeds_replace_bert_classifier.tokenizer.sep_token,
        input_embeds_replace_bert_classifier.tokenizer.pad_token,
        input_embeds_replace_bert_classifier.tokenize_function,
    )
)

In [8]:
vanilla_df = vanilla_bert_classifier.forward_dataset_and_save_outputs(
    dataset_vanilla,
    kg_manager.get_vanilla_tokens_as_df,
    epochs=1,
    batch_size=BATCH_SIZE,
    force_recompute=False,
)
prompt_df = prompt_bert_classifier.forward_dataset_and_save_outputs(
    dataset_prompt,
    kg_manager.get_prompt_tokens_as_df,
    epochs=1,
    force_recompute=False,
)
input_embeds_replace_df = (
    input_embeds_replace_bert_classifier.forward_dataset_and_save_outputs(
        dataset_input_embeds_replace,
        kg_manager.get_vanilla_tokens_as_df,
        epochs=1,
        batch_size=BATCH_SIZE,
        force_recompute=False,
    )
)

./data/llm/vanilla/attentions.npy ./data/llm/vanilla/hidden_states.npy ./data/llm/vanilla/tokens.csv
./data/llm/prompt/attentions.npy ./data/llm/prompt/hidden_states.npy ./data/llm/prompt/tokens.csv
./data/llm/input_embeds_replace/attentions.npy ./data/llm/input_embeds_replace/hidden_states.npy ./data/llm/input_embeds_replace/tokens.csv


In [9]:
input_embeds_replace_df

,source_id,target_id,title,genres,labels,split,hidden_states,attentions
0,0,5,heat ( 1995 ),"['action ', 'crime ', 'thriller']",1,train,"[[[0.9417381, 0.11100921, -8.244461, 0.0768013...","[[[0.16464299, 0.058689594], [0.1322025, 0.042..."
1,0,89,bottle rocket ( 1996 ),"['adventure ', 'comedy ', 'crime ', 'romance']",1,train,"[[[0.9417381, 0.11100921, -8.244461, 0.0768013...","[[[0.087070346, 0.08129981], [0.057338703, 0.0..."
2,0,97,braveheart ( 1995 ),"['action ', 'drama ', 'war']",1,train,"[[[0.9417381, 0.11100921, -8.244461, 0.0768013...","[[[0.18892497, 0.073472254], [0.15924668, 0.03..."
3,0,184,billy madison ( 1995 ),['comedy'],1,train,"[[[0.9417381, 0.11100921, -8.244461, 0.0768013...","[[[0.13144213, 0.07449591], [0.10240837, 0.041..."
4,0,224,star wars : episode iv - a new hope ( 1977 ),"['action ', 'adventure ', 'sci - fi']",1,train,"[[[0.9417381, 0.11100921, -8.244461, 0.0768013...","[[[0.1672216, 0.03307326], [0.13944641, 0.0127..."
...,...,...,...,...,...,...,...,...
125032,79,8846,lovesick ( 2014 ),"['comedy ', 'romance']",0,val,"[[[0.9417381, 0.11100921, -8.244461, 0.0768013...","[[[0.084072135, 0.103651546], [0.031487536, 0...."
125033,300,2595,dersu uzala ( 1975 ),"['adventure ', 'drama']",0,val,"[[[0.9417381, 0.11100921, -8.244461, 0.0768013...","[[[0.06681271, 0.023924373], [0.022649713, 0.0..."
125034,321,59,lawnmower man 2 : beyond cyberspace ( 1996 ),"['action ', 'sci - fi ', 'thriller']",0,val,"[[[0.9417381, 0.11100921, -8.244461, 0.0768013...","[[[0.11401038, 0.042020373], [0.033344958, 0.0..."
125035,388,2639,all the vermeers in new york ( 1990 ),"['comedy ', 'drama ', 'romance']",0,val,"[[[0.9417381, 0.11100921, -8.244461, 0.0768013...","[[[0.049182758, 0.040349208], [0.017131926, 0...."


In [10]:
dataset = kg_manager.generate_huggingface_dataset(
    [vanilla_df, prompt_df, input_embeds_replace_df],
    ["vanilla", "prompt", "input_embeds_replace"],
)

In [11]:
dataset.save_to_disk("./data/dataset.hf")

Saving the dataset (0/7 shards):   0%|          | 0/56469 [00:00<?, ? examples/s]

Saving the dataset (0/4 shards):   0%|          | 0/34284 [00:00<?, ? examples/s]

Saving the dataset (0/4 shards):   0%|          | 0/34284 [00:00<?, ? examples/s]

In [12]:
dataset.push_to_hub("AhmadPython/MovieLens_KGE")

Uploading the dataset shards:   0%|          | 0/7 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/9 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/9 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/9 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/9 [00:00<?, ?ba/s]

...

Creating parquet from Arrow format:   0%|          | 0/9 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/9 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/2.99k [00:00<?, ?B/s]

c:\Users\MARS\.conda\envs\hauptprojekt\Lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\MARS\.cache\huggingface\hub\datasets--AhmadPython--MovieLens_KGE. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


CommitInfo(commit_url='https://huggingface.co/datasets/AhmadPython/MovieLens_KGE/commit/60a2bc61bdf043539b0e3091c46ba6bd0fa0c931', commit_message='Upload dataset', commit_description='', oid='60a2bc61bdf043539b0e3091c46ba6bd0fa0c931', pr_url=None, pr_revision=None, pr_num=None)

In [13]:
dataset

DatasetDict({
    train: Dataset({
        features: ['source_id', 'target_id', 'id_x', 'id_y', 'prompt_feature_title', 'prompt_feature_genres', 'labels', 'split', 'prompt', 'gnn_feature_(no genres listed)', 'gnn_feature_Action', 'gnn_feature_Adventure', 'gnn_feature_Animation', 'gnn_feature_Children', 'gnn_feature_Comedy', 'gnn_feature_Crime', 'gnn_feature_Documentary', 'gnn_feature_Drama', 'gnn_feature_Fantasy', 'gnn_feature_Film-Noir', 'gnn_feature_Horror', 'gnn_feature_IMAX', 'gnn_feature_Musical', 'gnn_feature_Mystery', 'gnn_feature_Romance', 'gnn_feature_Sci-Fi', 'gnn_feature_Thriller', 'gnn_feature_War', 'gnn_feature_Western', 'vanilla_attentions', 'vanilla_hidden_states', 'vanilla_attentions_original_shape', 'vanilla_hidden_states_original_shape', 'prompt_attentions', 'prompt_hidden_states', 'prompt_attentions_original_shape', 'prompt_hidden_states_original_shape', 'input_embeds_replace_attentions', 'input_embeds_replace_hidden_states', 'input_embeds_replace_attentions_original